# Battery Thermal Runaway Digital Twin - PINN

Physics-informed neural network for 18650 Li-ion battery thermal prediction with uncertainty quantification.

**Reference:**  
Coman, P.T., Darcy, E.C., & White, R.E. (2022). Simplified Thermal Runaway Model for Assisting the Design of a Novel Safe Li-Ion Battery Pack. *J. Electrochem. Soc.* **169**, 040516.  
[https://doi.org/10.1149/1945-7111/ac62bd](https://doi.org/10.1149/1945-7111/ac62bd)

**Task:** Digital twin demonstrator for USC CIBI research group (Dr. Paul Coman)

## Setup

In [ ]:
# Clone repository and install dependencies
!git clone https://github.com/tsa2000/Battery-PINN.git
%cd Battery-PINN
!pip install -q -r requirements.txt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from src.model import BatteryThermalPINN
from src.physics import ThermalModel
from src.scenarios import ScenarioGenerator
from src.agent import BatteryDigitalTwin
from src.utils import generate_data, train_pinn, train_ensemble, predict_with_uncertainty

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

## 1. Data Generation

Generate training data using ODE solver with sparse measurements (simulating real sensor data)

In [ ]:
# Initialize thermal model (0D lumped capacitance)
thermal_model = ThermalModel()

# Print model parameters
print("Model Parameters:")
print(f"  Mass: {thermal_model.m*1000:.1f} g")
print(f"  Specific heat: {thermal_model.Cp} J/kg/K")
print(f"  Surface area: {thermal_model.A*1e4:.2f} cm²")
print(f"  Convection coeff: {thermal_model.h} W/m²/K")
print(f"  Peak heat rate: {thermal_model.Q_peak/1000:.1f} kW")

# Generate sparse training data (20 points with 5% noise)
t_data_sparse = np.linspace(0, 50, 20)
T_data = generate_data(thermal_model, t_data_sparse, T0=25, noise_level=0.05)

# Physics collocation points (for enforcing PDE)
t_physics = np.linspace(0, 50, 100)

# Plot training data
plt.figure(figsize=(8, 5))
plt.scatter(t_data_sparse, T_data, c='blue', s=50, label='Training Data (Noisy)', zorder=3)
plt.xlabel('Time (s)', fontsize=11)
plt.ylabel('Temperature (°C)', fontsize=11)
plt.title('Sparse Training Data (20 measurements)', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTraining data: {len(t_data_sparse)} points")
print(f"Physics points: {len(t_physics)} points")

## 2. Train Ensemble PINN

Train 5 models with different noise realizations for uncertainty quantification

In [ ]:
# Train ensemble (5 models for speed, can increase to 10 for better UQ)
n_models = 5
print(f"Training ensemble of {n_models} PINN models...\n")

ensemble_models = train_ensemble(
    n_models=n_models,
    thermal_model=thermal_model,
    t_data=t_data_sparse,
    T_data=T_data,
    t_physics=t_physics,
    epochs=3000,  # Reduced for demo (use 5000+ for better accuracy)
    noise_levels=[0.0, 0.02, 0.05]
)

print(f"\n✓ Ensemble ready ({n_models} models)")

## 3. Predictions with Uncertainty

Compare PINN predictions against analytical solution

In [ ]:
# Generate test predictions
t_test = np.linspace(0, 50, 500)
T_mean, T_std = predict_with_uncertainty(ensemble_models, t_test)

# Ground truth (analytical solution)
T_analytical = generate_data(thermal_model, t_test, T0=25, noise_level=0.0)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(t_test, T_analytical, 'k--', label='Ground Truth (ODE)', linewidth=2, alpha=0.7)
plt.plot(t_test, T_mean, 'r-', label='PINN Prediction', linewidth=2)
plt.fill_between(t_test, T_mean - 2*T_std, T_mean + 2*T_std, 
                 alpha=0.3, color='red', label='95% Confidence')
plt.scatter(t_data_sparse, T_data, c='blue', s=40, label='Training Data', zorder=5)
plt.axhline(60, color='orange', linestyle=':', linewidth=2, label='TR Onset (~60°C)')
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Temperature (°C)', fontsize=12)
plt.title('Thermal Runaway Prediction with Uncertainty Quantification', fontsize=13)
plt.legend(loc='best', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Model Performance Metrics

In [ ]:
# Calculate error metrics
mae = np.mean(np.abs(T_mean - T_analytical))
rmse = np.sqrt(np.mean((T_mean - T_analytical)**2))
mape = np.mean(np.abs((T_mean - T_analytical) / (T_analytical + 1e-8))) * 100

# Check 95% confidence interval coverage
lower = T_mean - 2*T_std
upper = T_mean + 2*T_std
coverage = np.mean((T_analytical >= lower) & (T_analytical <= upper)) * 100

print("=" * 55)
print("MODEL PERFORMANCE METRICS")
print("=" * 55)
print(f"Mean Absolute Error (MAE):         {mae:.3f} °C")
print(f"Root Mean Square Error (RMSE):     {rmse:.3f} °C")
print(f"Mean Absolute Percentage Error:    {mape:.2f}%")
print(f"95% CI Coverage:                   {coverage:.1f}%")
print("=" * 55)

if mape < 5.0:
    print("✓ Good accuracy for 0D lumped model")
else:
    print(f"Note: {mape:.1f}% error acceptable for simplified model")

## 5. Physics Residual Check

Verify that PINN satisfies the governing PDE

In [ ]:
# Compute physics residual for first model
t_check = np.linspace(0, 50, 200)
residual = thermal_model.physics_residual(ensemble_models[0], t_check)

plt.figure(figsize=(10, 4))
plt.plot(t_check, residual.detach().numpy(), linewidth=2, color='purple')
plt.axhline(0, color='red', linestyle='--', alpha=0.5, label='Perfect physics')
plt.xlabel('Time (s)', fontsize=11)
plt.ylabel('Residual', fontsize=11)
plt.title('Physics Constraint: m·Cp·dT/dt - Q_TR + Q_conv', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

residual_mean = np.abs(residual.detach().numpy()).mean()
print(f"Mean absolute residual: {residual_mean:.2e}")
print("(Small residual = model respects physics)")

## 6. Operating Scenarios Analysis

Test different realistic scenarios

In [ ]:
import copy

scenario_gen = ScenarioGenerator(thermal_model)
scenarios = scenario_gen.get_all_scenarios()

fig, axes = plt.subplots(1, 5, figsize=(18, 4))

for idx, scenario in enumerate(scenarios):
    # Create temporary model (avoid modifying original)
    tm_temp = copy.deepcopy(thermal_model)
    
    # Apply scenario modifications
    Q_scale = scenario.get('Q_scale', 1.0)
    tm_temp.Q_peak = thermal_model.Q_peak * Q_scale
    
    if 'Cp_modified' in scenario:
        tm_temp.Cp = scenario['Cp_modified']
    
    # Generate temperature profile
    t_scenario = np.linspace(0, 50, 300)
    T0 = scenario.get('T_initial', 25)
    T_amb = scenario.get('T_amb', 25)
    t_onset = scenario.get('t_onset', 10.0)
    
    T_scenario = generate_data(tm_temp, t_scenario, T0=T0, T_amb=T_amb, 
                                t_onset=t_onset, noise_level=0.0)
    
    # Plot
    axes[idx].plot(t_scenario, T_scenario, linewidth=2.5, color='C' + str(idx))
    axes[idx].axhline(60, color='red', linestyle='--', alpha=0.6, linewidth=1.5)
    axes[idx].set_title(scenario['name'], fontsize=10, fontweight='bold')
    axes[idx].set_xlabel('Time (s)', fontsize=9)
    axes[idx].set_ylabel('T (°C)', fontsize=9)
    axes[idx].grid(alpha=0.3)
    axes[idx].tick_params(labelsize=8)

plt.suptitle('Operating Scenarios Comparison', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Scenarios tested:")
for i, s in enumerate(scenarios, 1):
    print(f"  {i}. {s['name']}")

## 7. AI Query Agent Demo

Natural language interface for battery safety queries

In [ ]:
# Initialize digital twin agent
agent = BatteryDigitalTwin(ensemble_models, thermal_model)

# Example queries
queries = [
    "What happens if the vehicle cruises at 100 km/h for 10 minutes?",
    "Is it safe to fast-charge now?",
    "How does the battery behave in cold weather?",
    "What about cruising at 80 km/h for 15 minutes?"
]

print("=" * 75)
print("AI QUERY AGENT - BATTERY DIGITAL TWIN")
print("=" * 75)

for query in queries:
    print(f"\n🔹 Query: {query}")
    print("-" * 75)
    response = agent.query(query)
    print(response)
    print()

## 8. Custom Query (Interactive)

Try your own queries!

In [ ]:
# Example custom queries (uncomment to use interactive input)
custom_queries = [
    "cruise at 120 km/h for 8 minutes",
    "cruise at 60 km/h for 20 minutes"
]

print("\nCUSTOM QUERY EXAMPLES:")
print("=" * 75)
for q in custom_queries:
    print(f"\n🔹 Query: {q}")
    print(agent.query(q))

# For interactive use (works in local Jupyter, may not work in Colab presentation):
# user_query = input("\nYour query: ")
# print(agent.query(user_query))

## Summary

**What was demonstrated:**
1. ✓ 0D thermal model (Coman Eq. 1) with heat generation from Fig. 3
2. ✓ PINN surrogate with physics-informed training
3. ✓ Uncertainty quantification via ensemble (5 models)
4. ✓ Operating scenarios (fast-charge, cold, highway, etc.)
5. ✓ AI query agent for natural language interaction

**Limitations:**
- 0D model: no spatial temperature gradients
- Constant properties (Cp, h) - real cells show variation
- No ejecta or gas flow modeling
- Speed-to-C-rate mapping is empirical

**Future improvements:**
- Extend to 1D/2D for spatial resolution
- Include temperature-dependent properties
- Validate against experimental data from USC CIBI lab